In [1]:
import ee
import geemap.foliumap as geemap

In [2]:
ee.Authenticate()

Enter verification code: 4/1AbUR2VMreyK0Cv2RwqCaxf8mQR7ZJ1y_SvhgCaInhTOnA7AKTECRmI-I0s8

Successfully saved authorization token.


In [3]:
ee.Initialize()

In [5]:
zone = ee.FeatureCollection("users/nachogbn/cara_norte")

In [7]:
m = geemap.Map()
m.centerObject(zone)
m.addLayer(zone)
m

In [32]:
bands = []

years = range(2000, 2021)
for year in years:
    str_year = str(year)
    l7collection = (
        ee.ImageCollection("LANDSAT/LE07/C01/T1_SR")
        .filterBounds(zone)
        .filterDate(str_year + "-01-01", str_year + "-12-31")
    )

    l7ndvi = l7collection.map(get_NDVI)

    composite = l7ndvi.qualityMosaic("NDVI").clip(zone)

    band = composite.select("NDVI")

    bands.append(band)

In [33]:
m = geemap.Map()

m.centerObject(zone, zoom=10)

vis_params = {"min": 0, "max": 1, "palette": ["beb261", "8ffff9", "1cd810"]}

for i, band in enumerate(bands):
    m.addLayer(band, vis_params, "NDVI_" + str(2000 + i))

m.addLayerControl()

m

In [34]:
geemap.ee_export_image_to_drive(
    band, "band1", folder="GEE", region=zone.geometry(), scale=30
)

In [17]:
lat = 37.0311
lon = -3.3259
point = ee.Geometry.Point([lon, lat])

In [18]:
data = []
for band in bands:
    data.append(geemap.ee_to_numpy(band, region=point).item())
data

[0.27057576179504395,
 0.31987977027893066,
 0.29074665904045105,
 0.2624776363372803,
 0.28136172890663147,
 0.2567161023616791,
 0.25834619998931885,
 0.28746679425239563,
 0.2710568606853485,
 0.2651115357875824,
 0.2556403577327728,
 0.26921090483665466,
 0.2865734100341797,
 0.32375526428222656,
 0.2750408351421356,
 0.29840531945228577,
 0.3145057260990143,
 0.3133447766304016,
 0.3041315972805023,
 0.33869773149490356,
 0.3371117115020752]

In [3]:
import pandas as pd

df = pd.DataFrame(data)

df.plot()

NameError: name 'data' is not defined

In [4]:
import hvplot.pandas

df = pd.DataFrame(data)

df.hvplot()

NameError: name 'data' is not defined

In [5]:
df = pd.DataFrame({"NDVI": data}, index=range(2000, 2021))
df.index = df.index.rename("Year")
df.hvplot()

NameError: name 'data' is not defined

In [22]:
# Generate a single file with the different layers (bands)
stack_collection = ee.ImageCollection(bands)
ndvis = stack_collection.toBands()

In [23]:
import pathlib

desktop = pathlib.Path.home() / "Desktop"

videoArgs = {
    "dimensions": 768,
    "region": zone,
    "framesPerSecond": 1,
    "min": 0,
    "max": 1,
    "palette": ["beb261", "8ffff9", "1cd810"],
}

timelapse1 = desktop / "timelapse.gif"

geemap.download_ee_video(stack_collection, videoArgs, timelapse1)
geemap.show_image(timelapse1)

Generating URL...
Please wait ...
The GIF image has been saved to: C:\Users\Ignacio\Desktop\timelapse.gif


Output()

In [24]:
timelapse2 = desktop / "timelapse2.gif"

text = ["Año " + str(i) for i in range(2000, 2021)]

geemap.add_text_to_gif(
    timelapse1,
    timelapse2,
    xy=("3%", "5%"),
    text_sequence=text,
    font_size=30,
    font_color="#ffffff",
)

geemap.show_image(timelapse2)

Output()

In [25]:
ndvi_trend = ndvis.reduce(ee.Reducer.kendallsCorrelation())
tau = ndvi_trend.select("tau")

In [26]:
m = geemap.Map()

m.centerObject(zone, zoom=10)

vis_params = {"min": -1, "max": 1, "palette": ["red", "orange", "white", "green"]}

m.addLayer(tau, vis_params, "Mann-Kendal")

m

In [27]:
geemap.ee_export_image_to_drive(
    tau, "tau", folder="GEE", region=zone.geometry(), scale=30
)